In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.transforms import Normalize
import numpy as np
import os, glob
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import json
from scipy.ndimage import gaussian_filter
import scipy

In [ ]:
class FluidForceCNN(nn.Module):
    def __init__(self):
        super(FluidForceCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # input shape: (B, 1, 64, 128)
            nn.ReLU(),

            nn.Conv2d(64, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 60, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(60, 55, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(55, 50, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(50, 45, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(45, 40, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(40, 35, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(35, 28, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(28, 16, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(16, 2, kernel_size=3, padding=1),  # final output layer
            nn.Tanh()
        )

    def forward(self, x):
        return self.conv_layers(x)

In [ ]:
def evaluate_shape(shape_np, model, device='cpu'):
    """
    Runs a (64, 128, 1) shape input through the model and returns the predicted output.

    Parameters:
        shape_np (np.ndarray): Input shape array of shape (64, 128, 1)
        model (nn.Module): Trained PyTorch model
        device (str): 'cpu' or 'cuda'

    Returns:
        output_np (np.ndarray): Output prediction, shape (64, 128, 3)
    """
    model.eval()
    model.to(device)

    # Convert input: (64,128,1) → (1,1,64,128)
    input_tensor = torch.tensor(shape_np.transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        output_tensor = model(input_tensor)  # (1, 3, 64, 128)

    # Convert to NumPy: (1,3,64,128) → (64,128,3)
    output_np = output_tensor.cpu().numpy().squeeze(0).transpose(1, 2, 0)

    return output_np

In [ ]:
model = FluidForceCNN()
model.load_state_dict(torch.load("./models/best_model.pth"))
model.eval()